<a href="https://colab.research.google.com/github/caseyhyoon/W266-Final-Project/blob/casey/bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
import numpy as np
import pandas as pd
import os
import re
from sklearn.model_selection import train_test_split
import string
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [3]:
os.listdir()

['.config',
 'twitter_sexism_parsed_dataset.csv',
 'twitter_racism_parsed_dataset.csv',
 'twitter_parsed_dataset.csv',
 'sample_data']

In [4]:
parsed = pd.read_csv('twitter_parsed_dataset.csv')
racism = pd.read_csv('twitter_racism_parsed_dataset.csv')
sexism = pd.read_csv('twitter_sexism_parsed_dataset.csv')

twitter_data = pd.concat([parsed, racism, sexism]).dropna()
twitter_data.head()

,index,id,Text,Annotation,oh_label
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0


In [5]:
### Cleaning tweets

def cleaning_tweets(tweet):
    # 1. Remove Twitter handles (@user)
    users = re.findall("@[\w]*", tweet) # tokenizing
    for user in users:
        tweet = re.sub(user, '', tweet)
        
    # 2. Remove urls
    tweet = re.sub(r'http\S+', '', tweet)

    # 3. Remove, Punctuations, Numbers, and Special Characters (keep hashtags)
    tweet = tweet.replace(".", " ").replace(",", " ").replace("?", " ").replace("!", " ")
    tweet = "".join([char for char in tweet if char not in string.punctuation])
    tweet = re.sub('[0-9]+', '', tweet)

    # 4. Lowercase all
    tweet = tweet.lower()
    
    return tweet

twitter_data['cleaned_tweets'] = twitter_data['Text'].apply(cleaning_tweets)
twitter_data.head()

,index,id,Text,Annotation,oh_label,cleaned_tweets
0,5.74948705591165E+017,5.74948705591165E+017,@halalflaws @biebervalue @greenlinerzjm I read...,none,0.0,i read them in context no change in meaning...
1,5.71917888690393E+017,5.71917888690393E+017,@ShreyaBafna3 Now you idiots claim that people...,none,0.0,now you idiots claim that people who tried to...
2,3.90255841338601E+017,3.90255841338601E+017,"RT @Mooseoftorment Call me sexist, but when I ...",sexism,1.0,rt call me sexist but when i go to an auto p...
3,5.68208850655916E+017,5.68208850655916E+017,"@g0ssipsquirrelx Wrong, ISIS follows the examp...",racism,1.0,wrong isis follows the example of mohammed a...
4,5.75596338802373E+017,5.75596338802373E+017,#mkr No No No No No No,none,0.0,mkr no no no no no no


In [ ]:
X = twitter_data['cleaned_tweets']
y = np.array(twitter_data['oh_label'])

sum(y)/len(y)

0.23660862446622563

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(twitter_data['cleaned_tweets'], twitter_data['oh_label'], test_size = 0.2, random_state=1)


Run for subset of data

In [6]:
twitter_subset = twitter_data.sample(10000)
x_subset = twitter_subset['cleaned_tweets']
y_subset = twitter_subset['oh_label']

X_train, X_test, y_train, y_test = train_test_split(x_subset, y_subset, test_size = 0.2, random_state=1)

In [ ]:
baseline = 1 - np.mean(y_train)
baseline

0.7541249999999999

The dataset is imbalanced, we are going to under-sample the data for balanced training.

In [ ]:
good = twitter_data[twitter_data['oh_label'] == 0]
racist_sexist = twitter_data[twitter_data['oh_label'] == 1]

good_undersample = good.sample(int(sum(y)))
balanced_data = pd.concat([good_undersample, racist_sexist], axis=0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(balanced_data['cleaned_tweets'], balanced_data['oh_label'], test_size = 0.2, random_state=1)

X_train

4711     as expected  when the terrorist group hamas wo...
10126                                                here 
4797      in all seriousness  ive been dying my hair si...
13621    rt   just so i’m clear you have dogs named leo...
8004       kat you are the biggest mole   i hope you ch...
                               ...                        
805                       kat is the daughter of satan mkr
8717      wrong  apostacy is the equivalent of leaving ...
10392     its obvious why the former president of the n...
4754      can you explain the wage gap   what does the ...
8440                kat and andre are the fuckin devil mkr
Name: cleaned_tweets, Length: 17110, dtype: object

In [7]:
pip install transformers

     |████████████████████████████████| 2.2MB 5.2MB/s 
     |████████████████████████████████| 870kB 26.5MB/s 
     |████████████████████████████████| 3.3MB 37.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=582735c69a78d182031467114375ccd8c05c06c8ee2bb676d5382e26fcf57ade
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [8]:
from transformers import BertTokenizer, TFBertModel

In [9]:
bert_layer = TFBertModel.from_pretrained('bert-base-uncased')

Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Freezing layers

In [30]:
retrain_layers = []

for retrain_layer_number in range(4):

    layer_code = '_' + str(11 - retrain_layer_number)
    retrain_layers.append(layer_code)

for w in bert_layer.weights:
    if not any([x in w.name for x in retrain_layers]):
        w._trainable = False

In [31]:
tf.keras.backend.clear_session()

In [36]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

bert_inputs = tokenizer(list(X_train), padding=True, return_tensors='tf')
dim = bert_inputs['input_ids'].shape[1]

In [37]:
input_ids = tf.keras.layers.Input(shape=(dim), dtype='int32', name='input_ids')
masks = tf.keras.layers.Input(shape=(dim), dtype='int32', name='mask')
token_type_ids = tf.keras.layers.Input(shape=(dim), dtype='int32', name='token_types')

bert_output = bert_layer([input_ids, masks, token_type_ids])


cls = bert_output[1]

hidden = tf.keras.layers.Dense(200, activation='relu')(cls)

classification = tf.keras.layers.Dense(1, activation='sigmoid')(hidden)

model = tf.keras.Model(inputs = [input_ids, masks, token_type_ids], outputs = classification)
model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              optimizer=tf.keras.optimizers.Adam(lr=0.0005, beta_1=0.91, beta_2=0.999, epsilon=None, decay=0.1, amsgrad=False),
              # optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics='acc')

In [38]:
model.fit(x = [np.array(bert_inputs['input_ids']), np.array(bert_inputs['attention_mask']), np.array(bert_inputs['token_type_ids'])],
          y = y_train,
          epochs = 5,
          validation_split = 0.2,
          batch_size = 64)


Epoch 1/5
100/100 [==============================] - 48s 430ms/step - loss: 0.5027 - acc: 0.7814 - val_loss: 0.3420 - val_acc: 0.8587
Epoch 2/5
100/100 [==============================] - 41s 415ms/step - loss: 0.3691 - acc: 0.8359 - val_loss: 0.3310 - val_acc: 0.8656
Epoch 3/5
100/100 [==============================] - 41s 415ms/step - loss: 0.3137 - acc: 0.8706 - val_loss: 0.3118 - val_acc: 0.8706
Epoch 4/5
100/100 [==============================] - 42s 416ms/step - loss: 0.2834 - acc: 0.8854 - val_loss: 0.3041 - val_acc: 0.8838
Epoch 5/5
100/100 [==============================] - 42s 416ms/step - loss: 0.2278 - acc: 0.9103 - val_loss: 0.3006 - val_acc: 0.8869


In [39]:
bert_test_inputs = tokenizer(list(X_test), padding='max_length', truncation=True, max_length = dim, return_tensors='tf')
bert_test_inputs

{'input_ids': <tf.Tensor: shape=(2000, 44), dtype=int32, numpy=
array([[  101,  2027,  2439, ...,     0,     0,     0],
       [  101,  2327, 17710, ...,     0,     0,     0],
       [  101,  2236, 12455, ...,     0,     0,     0],
       ...,
       [  101, 19387,  2023, ...,     0,     0,     0],
       [  101,  2057,  2342, ...,     0,     0,     0],
       [  101,  3342, 23332, ...,     0,     0,     0]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(2000, 44), dtype=int32, numpy=
array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2000, 44), dtype=int32, numpy=
array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]], dtype=int32

In [45]:
y_pred = model.predict([np.array(bert_test_inputs['input_ids']), np.array(bert_test_inputs['attention_mask']), np.array(bert_test_inputs['token_type_ids'])])
y_pred = np.array([1 if x[0] >= 0.5 else 0  for x in y_pred])

print("accuracy: " + str(accuracy_score(y_test, y_pred)))

accuracy: 0.874


In [75]:
errors = pd.DataFrame((y_pred == y_test))
error_index = np.array(errors[errors['oh_label'] == False].index)

for error in error_index[:15]:
  print(twitter_data.iloc[error]['Text'], twitter_data.iloc[error]['oh_label'])
  print()

.@eaballesty @dileted @tim_geelen Those women are not exploited.  They make a calculated choice to use sex to advance careers. 1.0

@IamStan is bitlocker any good? i've never used it. didn't realize i could gpg an entire drive, either. :) 0.0

@AMohedin My 2:282 says that you can replace the testimony of one man with that of two women. 1.0

@dankmtl 1400 years ago my ass. That was the beginning and it has never stopped. It is an ingrained part of Islam. 1.0

Colin was pretty nice to the girls. Do you think he's giving them the dick? #MKR 1.0

@mykitchenrules time! THE TEXAN IS BACK BABY! #mykitchenrules #mkr 0.0

@KaylinEvergreen Twitter_ebooks on github 0.0

RT @travisnashh: @YesYoureSexist because you hate a group doesn't make you sexist dumbass 1.0

@LifeInKhilafah That just means that the only kinds of Europeans you will get are crazy. Other places have the brainwashed and uneducated. 0.0

@Jizyacollector @AbuMusabAlTurki @islamofascist_ @_ProtocolShami True about RT.  However, the

In [65]:
twitter_data.iloc[error_index[0]]

index                                         5.60847759482556E+017
id                                            5.60847759482556E+017
Text              .@eaballesty @dileted @tim_geelen Those women ...
Annotation                                                   sexism
oh_label                                                          1
cleaned_tweets        those women are not exploited   they make ...
Name: 11143, dtype: object

Strong possibility that BERT is just calling for one class? Why is that? There is a 25% imbalance in the training data.